# Install Library

In [1]:
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.8 MB/s eta 0:00:00


# Login HF

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device

cuda


device(type='cuda')

# Download Data

In [4]:
from datasets import load_dataset

data = load_dataset("Helsinki-NLP/multiun", "ar-en")

# Big Data --> "Helsinki-NLP/multiun", "ar-en"
# Big Data --> "Helsinki-NLP/un_pc", "ar-en"
# Mediam Data --> "Helsinki-NLP/opus-100", "ar-en"

data

Generating train split:   0%|          | 0/9759125 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 9759125
    })
})

# Slice Data From 9759125 to 50000

In [5]:
n_rows = 50000

# Select the first n_rows from the 'train' split
data['train'] = data['train'].select(range(n_rows))

In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 50000
    })
})

In [7]:
data = data['train'].train_test_split(test_size=0.2)

In [8]:
data['train'][0]

{'translation': {'ar': 'اعتمد مشروع القرار A/59/L.65 (القرار 59/312).',
  'en': 'Draft resolution A/59/L.65 was adopted (resolution 59/312).'}}

# Loading Model

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# checkpoint = "Helsinki-NLP/opus-mt-tc-big-en-ar"
checkpoint = 'Helsinki-NLP/opus-mt-en-ar'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
source_lang = "en"
target_lang = "ar"
prefix = "translate Arabic to English: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [11]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Loading CollectionSeq2Seq

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-07-05 23:52:47.797168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 23:52:47.797332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 23:52:47.926458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
import evaluate

metric = evaluate.load("sacrebleu")

In [14]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Loading Model

In [15]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Building Arguments

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./Helsinki-mt-ar-en",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=1,
    fp16=True,
    warmup_steps=2000,
    logging_steps=2000
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train THe Model

In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1000,1.229600
2000,1.162600
3000,1.160800
4000,1.130600
5000,1.136500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=5000, training_loss=1.1640099609375, metrics={'train_runtime': 560.4994, 'train_samples_per_second': 71.365, 'train_steps_per_second': 8.921, 'total_flos': 760732275179520.0, 'train_loss': 1.1640099609375, 'epoch': 1.0})

# Testing The Model

In [29]:
text = tokenized_data['train'][2]['translation']['en']

tokenized_data['train'][2]['translation']

{'ar': 'والهيئات التالية تمارس أنشطتها منذ عدة سنوات: فريق آسيا والمحيط الهادئ المعني بغسل الأموال (28 ولاية قضائية)، وفرقة العمل الكاريبية المعنيــة بالاجـراءات الماليـة (30 ولاية قضائية)، وفرقة العمل للإجراءات المالية لمكافحة غسل الأموال في أمريكـا اللاتينية (9 ولايات قضائية)، ولجنة الخبراء المختارة لتقييم تدابير مكافحة غسل الأموال التابعة لمجلس أوروبا (Monyval) (26 ولاية قضائية)، وفريق أفريقيا الشرقية والجنوبية المعنـي بمكافحــة غســل الأمــوال (14 ولاية قضائية).',
 'en': 'The following have been working for several years: the Asia-Pacific Group on Money Laundering (28 jurisdictions), the Caribbean Financial Action Task Force (30 jurisdictions), the Financial Action Task Force of South America against Money Laundering (9 jurisdictions), the Select Committee of Experts on the Evaluation of Anti-Money Laundering Measures of the Council of Europe (Moneyval) (26 jurisdictions) and the Eastern and Southern Africa Anti-Money Laundering Group (14 jurisdictions).'}

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/Helsinki-mt-ar-en/checkpoint-3000")
inputs = tokenizer(text, return_tensors="pt").input_ids

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [21]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/Helsinki-mt-ar-en/checkpoint-3000")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [22]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'اعتُمد مشروع القرار A/59/L.65 (القرار 59/312).'

In [32]:
def predict(text, model_checkpoint):
    print('Input: ', text, '\n\n')
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    inputs = tokenizer(text, return_tensors="pt").input_ids
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
    outputs = model.generate(inputs, max_new_tokens=300, do_sample=True, top_k=30, top_p=0.95)
    return "Output:  " + tokenizer.decode(outputs[0], skip_special_tokens=True)

In [33]:
predict(text, '/kaggle/working/Helsinki-mt-ar-en/checkpoint-5000')

Input:  The following have been working for several years: the Asia-Pacific Group on Money Laundering (28 jurisdictions), the Caribbean Financial Action Task Force (30 jurisdictions), the Financial Action Task Force of South America against Money Laundering (9 jurisdictions), the Select Committee of Experts on the Evaluation of Anti-Money Laundering Measures of the Council of Europe (Moneyval) (26 jurisdictions) and the Eastern and Southern Africa Anti-Money Laundering Group (14 jurisdictions). 




'Output:  وتعمل الهيئات التالية منذ عدة سنوات: فريق آسيا والمحيط الهادئ المعني بغسل الأموال (28 ولاية قضائية)، وفرقة العمل الكاريبية للإجراءات المالية (30 ولاية قضائية)، وفرقة العمل المعنية بالإجراءات المالية في أمريكا الجنوبية لمكافحة غسل الأموال (9 ولايات قضائية)، ولجنة الخبراء المختارة المعنية بتقييم تدابير مكافحة غسل الأموال التابعة لمجلس أوروبا (صندوق النقد الدولي) (26 ولاية قضائية)، وفريق أفريقيا الشرقية والجنوبية لمكافحة غسل الأموال (14 ولاية قضائية).'